In [ ]:
# Dependencies
import json
import requests
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import gmaps
import plotly.express as px
from scipy.stats import linregress
from urllib.request import urlopen
from scipy import stats
from config import apiKey
from sodapy import Socrata

# Use open client to obtain data
client = Socrata("data.cdc.gov", apiKey)
results = client.get("dnhi-s2bf", limit = 1000000)

# Convert to pandas DataFrame
death_quarterly = pd.DataFrame.from_records(results)

In [ ]:
# Cleaning columns and dropping NA values
death_quarterly_1 = death_quarterly.drop(columns=['footnote'])
death_quarterly_2 = death_quarterly_1.dropna(subset =['covid_19_deaths','total_deaths'])

In [ ]:
# Setting numerical values to int for future calculations
death_quarterly_2['total_deaths'] = death_quarterly_2['total_deaths'].astype(int)
death_quarterly_2['covid_19_deaths'] = death_quarterly_2['covid_19_deaths'].astype(int)

#Group by county level
death_gb = death_quarterly_2.groupby(['fips_code'])
death_gb_df = death_gb.sum()

In [ ]:
# Calculating new column to calculate covid death %
death_gb_df['covid_portion_of_total_deaths'] = death_gb_df['covid_19_deaths']/death_gb_df['total_deaths']
death_gb_df = pd.DataFrame(death_gb_df)
death_gb_df.reset_index(inplace=True)

In [ ]:
# Generating stats on the datafame to get range for plot
death_gb_df.describe(include='all')

In [ ]:
# URL for GET requests to retrieve county data
url = "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
response = requests.get(url).json()

In [ ]:
# Creating map of polygons representing county boundaries
fig = px.choropleth(death_gb_df, geojson=response, locations='fips_code', color='covid_portion_of_total_deaths',
                           color_continuous_scale="ylgnbu",
                           range_color=(0, .35),
                           scope="usa",
                           labels={'covid_portion_of_total_deaths':'Covid Deaths as % of Total Deaths'}
                          )
fig.show()

In [ ]:
# Renaming column for later merge
death_df = death_gb_df.rename(columns = {'fips_code':'fips'})

In [ ]:
# Use open client to obtain data - can take up to 5 minutes to get results in dataframe ~1.6M rows
client = Socrata("data.cdc.gov", apiKey)
results = client.get("8xkx-amqh", limit = 2000000)
                     
# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [ ]:
# Limiting to specific columns
results_df_columns = results_df[['date','fips','recip_state','recip_county',
                                     'completeness_pct','administered_dose1_recip',
                                     'administered_dose1_pop_pct','series_complete_yes',
                                     'series_complete_pop_pct','metro_status','census2019']]

In [ ]:
# Setting up a clean datafame with out any NA and setting a date column
clean_df = results_df_columns.dropna()
clean_df['date'] = pd.to_datetime(clean_df['date']).dt.date

In [ ]:
# Inserting Year Quarter Month columns
clean_df['month'] = pd.DatetimeIndex(clean_df['date']).month
clean_df['quarter'] = pd.DatetimeIndex(clean_df['date']).quarter
clean_df['year'] = pd.DatetimeIndex(clean_df['date']).year

In [ ]:
# Grouping by county and taking max value
grouped_fips = clean_df.groupby(['fips'])
max_group = grouped_fips.max()

In [ ]:
# Creating a data frame by vaccine dose
max_dose = pd.DataFrame(max_group[['administered_dose1_recip','administered_dose1_pop_pct']]) 

In [ ]:
# Merging Death dataframe and Vaccine data frame
merged_df = death_df.merge(max_dose,on='fips')

In [ ]:
# Renaming columns from deaths dataframe and assigning proper types to columns
death_quarterly_2 = death_quarterly.rename(columns = {'fips_code' : 'fips'})
death_quarterly_2['year']= death_quarterly_2['year'].astype(int)
death_quarterly_2['quarter']=death_quarterly_2['quarter'].astype(int)
clean_df['year']=clean_df['year'].astype(int)
clean_df['quarter']=clean_df['quarter'].astype(int)

In [ ]:
# Merging deaths and our clean dataframe and limiting to 4 states CA,FL,TX,NY
total_quarter = pd.merge(death_quarterly_2,clean_df, on = ('fips','year','quarter'),how ='inner')
total_quarter_2 = total_quarter.loc[(total_quarter['state_of_residence']=='Florida')|(total_quarter['state_of_residence'] =='Texas')|(total_quarter['state_of_residence'] =='California')|(total_quarter['state_of_residence'] =='New York'),:]

In [ ]:
# Inserting quarter column for plotting purposes
total_quarter_2['quarter_number'] = pd.to_datetime(total_quarter_2['year'].astype(str) + 'Q' + total_quarter_2['quarter'].astype(str))

In [ ]:
# Cleaning out any NA from merge and calculating covid death rate
total_quarter_2.dropna(subset = ['covid_19_deaths','total_deaths'])
total_quarter_2['covid_death_rate']= total_quarter_2['covid_19_deaths'].astype(float)/total_quarter_2['total_deaths'].astype(float)

In [ ]:
# Removing duplicate values
total_quarter_2.drop_duplicates(subset =['quarter_number','fips'],inplace=True)                                                                                                                                    

In [ ]:
# Column names to use for plots
total_quarter_2.columns

In [ ]:
# Setting proper variable types
total_quarter_2['administered_dose1_recip'] = total_quarter_2['administered_dose1_recip'].astype(float) 
total_quarter_2['census2019'] = total_quarter_2['census2019'].astype(float) 
total_quarter_2['covid_19_deaths'] = total_quarter_2['covid_19_deaths'].astype(float) 
total_quarter_2['total_deaths'] = total_quarter_2['total_deaths'].astype(float) 

In [ ]:
# Creating df for California
california_df = total_quarter_2.loc[total_quarter_2['state_of_residence'] == 'California',:]
cali_quarter_group = california_df.groupby('quarter_number')
cali_quarter_df = cali_quarter_group.sum()

In [ ]:
# Adding calculated columns for covid deaths and vaccine rate
cali_quarter_df['covid_death_rate']= cali_quarter_df['covid_19_deaths']/cali_quarter_df['total_deaths']
cali_quarter_df['vaccine_rate']= cali_quarter_df['administered_dose1_recip']/cali_quarter_df['census2019']


In [ ]:
# Setting proper variables for original quarter dataframe
total_quarter['administered_dose1_recip'] = total_quarter['administered_dose1_recip'].astype(float) 
total_quarter['census2019'] = total_quarter['census2019'].astype(float) 
total_quarter['covid_19_deaths'] = total_quarter['covid_19_deaths'].astype(float) 
total_quarter['total_deaths'] = total_quarter['total_deaths'].astype(float) 

In [ ]:
# Creating df for Florida
florida_df = total_quarter_2.loc[total_quarter_2['state_of_residence'] == 'Florida',:]
FL_quarter_group = florida_df.groupby('quarter_number')
FL_quarter_df = FL_quarter_group.sum()

In [ ]:
# Adding calculated columns for covid deaths and vaccine rate
FL_quarter_df['covid_death_rate']= FL_quarter_df['covid_19_deaths']/FL_quarter_df['total_deaths']
FL_quarter_df['vaccine_rate']= FL_quarter_df['administered_dose1_recip']/FL_quarter_df['census2019']

In [ ]:
# Creating df for Texas
texas_df = total_quarter_2.loc[total_quarter_2['state_of_residence'] == 'Texas',:]
TX_quarter_group = texas_df.groupby('quarter_number')
TX_quarter_df = TX_quarter_group.sum()

In [ ]:
# Adding calculated columns for covid deaths and vaccine rate
TX_quarter_df['covid_death_rate']= TX_quarter_df['covid_19_deaths']/TX_quarter_df['total_deaths']
TX_quarter_df['vaccine_rate']= TX_quarter_df['administered_dose1_recip']/TX_quarter_df['census2019']


In [ ]:
# Creating dataframe for scatter plot
scatter_quarter = total_quarter
scatter_quarter['quarter_number'] = pd.to_datetime(total_quarter['year'].astype(str) + 'Q' + total_quarter['quarter'].astype(str))

In [ ]:
# Limiting to specific columns
scatter_plot = scatter_quarter.loc[:,['fips','administered_dose1_recip','census2019','covid_19_deaths','total_deaths','quarter_number']]

In [ ]:
# Removing duplicates
scatter_plot.drop_duplicates(subset =['quarter_number','fips'],inplace=True)

In [ ]:
# Removing NA values
scatter_plot.dropna()

In [ ]:
# Setting a clean df for the scatter
clean_scatter = scatter_plot.loc[(scatter_plot['administered_dose1_recip'] != 0) & (scatter_plot['covid_19_deaths'] != 0) & (scatter_plot['census2019'] != 0) &(scatter_plot['total_deaths'] != 0),:]

In [ ]:
# Removing 0 values and inf values to get proper rate calculations
clean_scatter.replace([np.inf, -np.inf], np.nan, inplace=True)
clean_scatter.dropna(inplace=True)

In [ ]:
# Scatter plot to show covid death % and vaccine pop fraction
x_x = clean_scatter['administered_dose1_recip']/clean_scatter['census2019']
y_y = clean_scatter['covid_19_deaths']/clean_scatter['total_deaths']

fig1, ax1 = plt.subplots()
plt.scatter(x_x,y_y)
plt.xlabel('Fraction of Pop Vaccinated')
plt.ylabel('Covid Death %')
plt.show()

In [ ]:
# Linear regression on scatter plot
correlation = stats.pearsonr(x_x,y_y)
print(f"The correlation between vaccine percent \033[34mand\033[0m the covid death percent \033[34mis\033[0m \033[31m{round(correlation[0],2)}\033[0m")

# Creating Plot for scatter and adding line
(slope, intercept,rvalue, pvalue, stderr) = linregress(x_x,y_y)
regress = x_x* slope + intercept
line_eq = f"y = {round(slope, 2)} x + {round(intercept, 2)}"

plt.scatter(x_x,y_y, s = 3)
plt.plot(x_x, regress, color='red')
plt.annotate(line_eq,(.4,.5),fontsize=12,color="red")
plt.xlabel('Fraction of Pop Vaccinated')
plt.ylabel('Covid Death %')

# Uncomment below to save image to desired location
#plt.savefig("../Images/Scatter_Vaccine_vs_Death")

plt.show()


In [ ]:
# Creating 2D heatmap from scatter to show concentration
x_min = np.min(x_x)
x_max = np.max(x_x)
  
y_min = np.min(y_y)
y_max = np.max(y_y)

x_bins = np.linspace(x_min, x_max, 70)
y_bins = np.linspace(y_min, y_max, 50)

fig, ax = plt.subplots()
plt.hist2d(x_x, y_y, bins = [x_bins,y_bins],cmap = plt.cm.nipy_spectral)

plt.xlabel('Fraction of Pop Vaccinated')
plt.ylabel('Covid Death %')

# Uncomment below to save image to desired location
#plt.savefig("../Images/Scatter_Vaccine_vs_Death_Heat")

plt.show()

In [ ]:
# Creating df for New York
NY_df = total_quarter_2.loc[total_quarter_2['state_of_residence'] == 'New York',:]
NY_quarter_group = NY_df.groupby('quarter_number')
NY_quarter_df = NY_quarter_group.sum()

In [ ]:
# Adding calculated columns for covid deaths and vaccine rate
NY_quarter_df['covid_death_rate']= NY_quarter_df['covid_19_deaths']/NY_quarter_df['total_deaths']
NY_quarter_df['vaccine_rate']= NY_quarter_df['administered_dose1_recip']/NY_quarter_df['census2019']

In [ ]:
# Set index length and bar width
quarter_label = ['2020 Q4', '2021 Q1','2021 Q2']
index = np.arange(3)
bar_width = 0.20

# Plot COVID-19 death & Vaccine counts by State
fig, ax = plt.subplots()
ny = ax.bar(index,NY_quarter_df["covid_death_rate"], bar_width, label='COVID-19 Death Rate', color='red', alpha=0.5)
ny2 = ax.bar(index+bar_width,NY_quarter_df["vaccine_rate"], bar_width, label='Vaccine % of Total', color='green', alpha=0.5)

# Assign title, labels, and legend
ax.set_xlabel('Quarter')
ax.set_ylabel('Percent')
ax.set_title('New York COVID-19 Death Rate vs. Vaccine Percent')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(quarter_label, rotation=45, ha='right')
ax.legend()

# Display the chart, and export for later use
plt.tight_layout()

# Uncomment below to save image to desired location
#plt.savefig('../Images/NY_Covid_Death_vs_Vaccine_Rate.png')

plt.show()

In [ ]:
# Set index length and bar width
quarter_label = ['2020 Q4', '2021 Q1','2021 Q2']
index = np.arange(3)
bar_width = 0.20

# Plot COVID-19 death & Vaccine counts by State
fig, ax = plt.subplots()
ny = ax.bar(index,cali_quarter_df["covid_death_rate"], bar_width, label='COVID-19 Death Rate', color='red', alpha=0.5)
ny2 = ax.bar(index+bar_width,cali_quarter_df["vaccine_rate"], bar_width, label='Vaccine % of Total', color='green', alpha=0.5)

# Assign title, labels, and legend
ax.set_xlabel('Quarter')
ax.set_ylabel('Percent')
ax.set_title('California COVID-19 Death Rate vs. Vaccine Percent')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(quarter_label, rotation=45, ha='right')
ax.legend()

# Display the chart, and export for later use
plt.tight_layout()

# Uncomment below to save image to desired location
#plt.savefig('../Images/CA_Covid_Death_vs_Vaccine_Rate.png')

plt.show()

In [ ]:
# Set index length and bar width
quarter_label = ['2020 Q4', '2021 Q1','2021 Q2']
index = np.arange(3)
bar_width = 0.20

# Plot COVID-19 death & Vaccine counts by State
fig, ax = plt.subplots()
ny = ax.bar(index,FL_quarter_df["covid_death_rate"], bar_width, label='COVID-19 Death Rate', color='red', alpha=0.5)
ny2 = ax.bar(index+bar_width,FL_quarter_df["vaccine_rate"], bar_width, label='Vaccine % of Total', color='green', alpha=0.5)

# Assign title, labels, and legend
ax.set_xlabel('Quarter')
ax.set_ylabel('Percent')
ax.set_title('Florida COVID-19 Death Rate vs. Vaccine Percent')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(quarter_label, rotation=45, ha='right')
ax.legend()

# Display the chart, and export for later use
plt.tight_layout()

# Uncomment below to save image to desired location
#plt.savefig('../Images/FL_Covid_Death_vs_Vaccine_Rate.png')

plt.show()